In [1]:
import pandas as pd

In [24]:
data = pd.read_csv("sample_2.csv")

In [25]:
data

,id,user_id,timestamp,billing_period,billing_total_price_usd
0,23,1,1706555483,6,60


In [26]:
def process(df):
    df['start_date'] = pd.to_datetime(df['timestamp'], unit='s')
    df['duration_timedelta'] = pd.to_timedelta(df['billing_period'], unit='D')
    df['end_date'] = df['start_date'] + df['duration_timedelta']
    february_start = pd.Timestamp('2024-02-01')
    february_end = pd.Timestamp('2024-02-29')  # 2024 - високосный год
    df['active_in_february'] = (
        (df['start_date'] <= february_end) & (df['end_date'] >= february_start)
    )
    active_subscriptions = df[df['active_in_february']]
    active_subscriptions['daily_contribution'] = active_subscriptions['billing_total_price_usd'] / active_subscriptions['billing_period']
    active_subscriptions['overlap_start'] = active_subscriptions['start_date'].clip(lower=february_start)
    active_subscriptions['overlap_end'] = active_subscriptions['end_date'].clip(upper=february_end)
    active_subscriptions['active_days'] = (active_subscriptions['overlap_end'] - active_subscriptions['overlap_start']).dt.days + 1
    active_subscriptions['february_contribution'] = active_subscriptions['daily_contribution'] * active_subscriptions['active_days']
    top_contributions = active_subscriptions.groupby('user_id')['february_contribution'].sum().nlargest(3)
    return top_contributions.sum().round(2)



In [27]:
process(data)

40.0